# Ungraded Lab: Feature Engineering Pipeline


In this lab, you will continue exploring [Tensorflow Transform](https://www.tensorflow.org/tfx/transform/get_started). This time, it will be in the context of a machine learning (ML) pipeline. In production-grade projects, you want to streamline tasks so you can more easily improve your model or find issues that may arise. [Tensorflow Extended (TFX)](https://www.tensorflow.org/tfx) provides components that work together to execute the most common steps in a machine learning project. If you want to dig deeper into the motivations behind TFX and the need for machine learning pipelines, you can read about it in [this paper](https://storage.googleapis.com/pub-tools-public-publication-data/pdf/b500d77bc4f518a1165c0ab43c8fac5d2948bc14.pdf) and in this [blog post](https://blog.tensorflow.org/2020/09/brief-history-of-tensorflow-extended-tfx.html).

You will build end-to-end pipelines in future courses but for this one, you will only build up to the feature engineering part. Specifically, you will:

* ingest data from a base directory with `ExampleGen`
* compute the statistics of the training data with `StatisticsGen`
* infer a schema with `SchemaGen`
* detect anomalies in the evaluation data with `ExampleValidator`
* preprocess the data into features suitable for model training with `Transform`

If several steps mentioned above sound familiar, it's because the TFX components that deal with data validation and analysis (i.e. `StatisticsGen`, `SchemaGen`, `ExampleValidator`) uses [Tensorflow Data Validation (TFDV)](https://www.tensorflow.org/tfx/data_validation/get_started) under the hood. You're already familiar with this library from the exercises in Week 1 and for this week, you'll see how it fits within an ML pipeline.

The components you will use are the orange boxes highlighted in the figure below:

<img src='img/feature_eng_pipeline.png'>



## Setup

### Install Seoul bike sharing dataset
- Note that you have to remove the non utf-8 characters in the header row. 
- Alternative you can just use the extracted version in this repository under seoul-data/

In [32]:
# !wget https://archive.ics.uci.edu/static/public/560/seoul+bike+sharing+demand.zip -P seoul-data
# !unzip seoul-data/seoul+bike+sharing+demand.zip -d seoul-data && rm -rf seoul-data/seoul+bike+sharing+demand.zip

--2023-11-02 17:51:42--  https://archive.ics.uci.edu/static/public/560/seoul+bike+sharing+demand.zip
Resolving archive.ics.uci.edu (archive.ics.uci.edu)... 128.195.10.252
Connecting to archive.ics.uci.edu (archive.ics.uci.edu)|128.195.10.252|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: unspecified
Saving to: ‘seoul-data/seoul+bike+sharing+demand.zip’

seoul+bike+sharing+     [  <=>               ] 590.13K  1.67MB/s    in 0.3s    

2023-11-02 17:51:43 (1.67 MB/s) - ‘seoul-data/seoul+bike+sharing+demand.zip’ saved [604298]

Archive:  seoul-data/seoul+bike+sharing+demand.zip
 extracting: seoul-data/SeoulBikeData.csv  


### Import packages

Let's begin by importing the required packages and modules. In case you want to replicate this in your local workstation, we used *Tensorflow v2.6* and *TFX v1.3.0*.

In [ ]:
!pip install -U google-cloud-aiplatform "shapely<2"

In [1]:
import tensorflow as tf

from tfx import v1 as tfx

from tfx.orchestration.experimental.interactive.interactive_context import InteractiveContext
from google.protobuf.json_format import MessageToDict

import os
import pprint
pp = pprint.PrettyPrinter()

2023-11-02 17:37:18.519576: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2023-11-02 17:37:19.635768: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not find TensorRT


Using TensorFlow backend


### Preprocess the CSV to extract extra features
- Namely convert the DD/MM/YY format into Day, Month, Year, DayOfWeek columns.
- All nice ints!

In [42]:
import pandas as pd
import os 
df = pd.read_csv('seoul-data/SeoulBikeData.csv')
df.Date = pd.to_datetime(df.Date, dayfirst=True)
df['Day'] = df.Date.dt.day
df['Month'] = df.Date.dt.month
df['Year'] = df.Date.dt.year
df['DayOfWeek'] = df.Date.dt.weekday
os.mkdir('seoul-data/edit')
df.to_csv("seoul-data/edit/SeoulBikeData2.csv", index=False)

### Define paths

You will define a few global variables to indicate paths in the local workspace.

In [43]:
# location of the pipeline metadata store
_pipeline_root = './pipeline/'

# directory of the raw data files
_data_root = './seoul-data/edit'

# path to the raw training data
_data_filepath = os.path.join(_data_root, 'SeoulBikeData2.csv')

### Create the Interactive Context

In [44]:
# Initialize the InteractiveContext with a local sqlite file.
# If you leave `_pipeline_root` blank, then the db will be created in a temporary directory.
# You can safely ignore the warning about the missing config file.
context = InteractiveContext(pipeline_root=_pipeline_root)

## Run TFX components interactively

With that, you can now run the pipeline interactively. You will see how to do that as you go through the different components below.

### ExampleGen

You will start the pipeline with the [ExampleGen](https://www.tensorflow.org/tfx/guide/examplegen) component. This  will:

*   split the data into training and evaluation sets (by default: 2/3 train, 1/3 eval).
*   convert each data row into `tf.train.Example` format. This [protocol buffer](https://developers.google.com/protocol-buffers) is designed for Tensorflow operations and is used by the TFX components.
*   compress and save the data collection under the `_pipeline_root` directory for other components to access. These examples are stored in `TFRecord` format. This optimizes read and write operations within Tensorflow especially if you have a large collection of data.

Its constructor takes the path to your data source/directory. In our case, this is the `_data_root` path. The component supports several data sources such as CSV, tf.Record, and BigQuery. Since our data is a CSV file, we will use [CsvExampleGen](https://www.tensorflow.org/tfx/api_docs/python/tfx/components/CsvExampleGen) to ingest the data.

Run the cell below to instantiate `CsvExampleGen`.

In [45]:
# Instantiate ExampleGen with the input CSV dataset
example_gen = tfx.components.CsvExampleGen(input_base=_data_root)

# Execute the component
context.run(example_gen)

ExecutionResult(
    component_id: CsvExampleGen
    execution_id: 43
    outputs:
        examples: OutputChannel(artifact_type=Examples, producer_component_id=CsvExampleGen, output_key=examples, additional_properties={}, additional_custom_properties={}, _input_trigger=None)

In a notebook environment, it may be useful to examine a few examples of the data especially if you're still experimenting. Since the data collection is saved in [TFRecord format](https://www.tensorflow.org/tutorials/load_data/tfrecord), you will need to use methods that work with that data type. You will need to unpack the individual examples from the `TFRecord` file and format it for printing. Let's do that in the following cells:

In [46]:
# get the artifact object
artifact = example_gen.outputs['examples'].get()[0]

# print split names and uri
print(f'split names: {artifact.split_names}')
print(f'artifact uri: {artifact.uri}')
artifact_uri = artifact.uri

# Get the URI of the output artifact representing the training examples, which is a directory
train_uri = os.path.join(artifact_uri, 'Split-train')

# Get the list of files in this directory (all compressed TFRecord files)
tfrecord_filenames = [os.path.join(train_uri, name)
                      for name in os.listdir(train_uri)]

# Create a `TFRecordDataset` to read these files
dataset = tf.data.TFRecordDataset(tfrecord_filenames, compression_type="GZIP")

split names: ["train", "eval"]
artifact uri: ./pipeline/CsvExampleGen/examples/43


In [47]:
# Define a helper function to get individual examples
def get_records(dataset, num_records):
    '''Extracts records from the given dataset.
    Args:
        dataset (TFRecordDataset): dataset saved by ExampleGen
        num_records (int): number of records to preview
    '''
    
    # initialize an empty list
    records = []
    
    # Use the `take()` method to specify how many records to get
    for tfrecord in dataset.take(num_records):
        
        # Get the numpy property of the tensor
        serialized_example = tfrecord.numpy()
        
        # Initialize a `tf.train.Example()` to read the serialized data
        example = tf.train.Example()
        
        # Read the example data (output is a protocol buffer message)
        example.ParseFromString(serialized_example)
        
        # convert the protocol bufffer message to a Python dictionary
        example_dict = (MessageToDict(example))
        
        # append to the records list
        records.append(example_dict)
        
    return records

In [48]:
# Get 3 records from the dataset
sample_records = get_records(dataset, 3)

# Print the output
pp.pprint(sample_records)

[{'features': {'feature': {'Date': {'bytesList': {'value': ['MjAxNy0xMi0wMQ==']}},
                           'Day': {'int64List': {'value': ['1']}},
                           'DayOfWeek': {'int64List': {'value': ['4']}},
                           'Dew point temperature(degC)': {'floatList': {'value': [-17.6]}},
                           'Functioning Day': {'bytesList': {'value': ['WWVz']}},
                           'Holiday': {'bytesList': {'value': ['Tm8gSG9saWRheQ==']}},
                           'Hour': {'int64List': {'value': ['0']}},
                           'Humidity(%)': {'int64List': {'value': ['37']}},
                           'Month': {'int64List': {'value': ['12']}},
                           'Rainfall(mm)': {'floatList': {'value': [0.0]}},
                           'Rented Bike Count': {'int64List': {'value': ['254']}},
                           'Seasons': {'bytesList': {'value': ['V2ludGVy']}},
                           'Snowfall (cm)': {'floatList': {'value

Now that `ExampleGen` has finished ingesting the data, the next step is data analysis.

### StatisticsGen
The [StatisticsGen](https://www.tensorflow.org/tfx/guide/statsgen) component computes statistics over your dataset for data analysis, as well as for use in downstream components (i.e. next steps in the pipeline). As mentioned earlier, this component uses TFDV under the hood so its output will be familiar to you.

`StatisticsGen` takes as input the dataset we just ingested using `CsvExampleGen`.

In [49]:
# Instantiate StatisticsGen with the ExampleGen ingested dataset
statistics_gen = tfx.components.StatisticsGen(
    examples=example_gen.outputs['examples'])

# Execute the component
context.run(statistics_gen)

ExecutionResult(
    component_id: StatisticsGen
    execution_id: 44
    outputs:
        statistics: OutputChannel(artifact_type=ExampleStatistics, producer_component_id=StatisticsGen, output_key=statistics, additional_properties={}, additional_custom_properties={}, _input_trigger=None)

You can display the statistics with the `show()` method.

*Note: You can safely ignore the warning shown when running the cell below.*

In [50]:
# Show the output statistics
context.show(statistics_gen.outputs['statistics'])

### SchemaGen

The [SchemaGen](https://www.tensorflow.org/tfx/guide/schemagen) component also uses TFDV to generate a schema based on your data statistics. As you've learned previously, a schema defines the expected bounds, types, and properties of the features in your dataset.

`SchemaGen` will take as input the statistics that we generated with `StatisticsGen`, looking at the training split by default.

In [51]:
# Instantiate SchemaGen with the StatisticsGen ingested dataset
schema_gen = tfx.components.SchemaGen(
    statistics=statistics_gen.outputs['statistics'],
    )

# Run the component
context.run(schema_gen)

ExecutionResult(
    component_id: SchemaGen
    execution_id: 45
    outputs:
        schema: OutputChannel(artifact_type=Schema, producer_component_id=SchemaGen, output_key=schema, additional_properties={}, additional_custom_properties={}, _input_trigger=None)

In [52]:
# Visualize the schema
context.show(schema_gen.outputs['schema'])

,Type,Presence,Valency,Domain
Feature name,,,,
'Date',BYTES,required,,-
'Day',INT,required,,-
'DayOfWeek',INT,required,,-
'Dew point temperature(degC)',FLOAT,required,,-
'Functioning Day',STRING,required,,'Functioning Day'
'Holiday',STRING,required,,'Holiday'
'Hour',INT,required,,-
'Humidity(%)',INT,required,,-
'Month',INT,required,,-


,Values
Domain,
'Functioning Day',"'No', 'Yes'"
'Holiday',"'Holiday', 'No Holiday'"
'Seasons',"'Autumn', 'Spring', 'Summer', 'Winter'"


### ExampleValidator

The [ExampleValidator](https://www.tensorflow.org/tfx/guide/exampleval) component detects anomalies in your data based on the generated schema from the previous step. Like the previous two components, it also uses TFDV under the hood. 

`ExampleValidator` will take as input the statistics from `StatisticsGen` and the schema from `SchemaGen`. By default, it compares the statistics from the evaluation split to the schema from the training split.

In [53]:
# Instantiate ExampleValidator with the StatisticsGen and SchemaGen ingested data
example_validator = tfx.components.ExampleValidator(
    statistics=statistics_gen.outputs['statistics'],
    schema=schema_gen.outputs['schema'])

# Run the component.
context.run(example_validator)

ExecutionResult(
    component_id: ExampleValidator
    execution_id: 46
    outputs:
        anomalies: OutputChannel(artifact_type=ExampleAnomalies, producer_component_id=ExampleValidator, output_key=anomalies, additional_properties={}, additional_custom_properties={}, _input_trigger=None)

In [54]:
# Visualize the results
context.show(example_validator.outputs['anomalies'])

### Transform
The [Transform](https://www.tensorflow.org/tfx/guide/transform) component performs feature engineering for both training and serving datasets. It uses the [TensorFlow Transform](https://www.tensorflow.org/tfx/transform/get_started) library introduced in the first ungraded lab of this week.

`Transform` will take as input the data from `ExampleGen`, the schema from `SchemaGen`, as well as a module containing the preprocessing function.

In this section, you will work on an example of a user-defined Transform code. The pipeline needs to load this as a module so you need to use the magic command `%% writefile` to save the file to disk. Let's first define a few constants that group the data's attributes according to the transforms we will perform later. This file will also be saved locally.

In [55]:
# Set the constants module filename
_bike_constants_module_file = 'bike_constants.py'

In [56]:
%%writefile {_bike_constants_module_file}

# String data types that will be converted to indices
CATEGORICAL_FEATURE_KEYS = [
    'Functioning Day', 'Holiday', 'Seasons'
]

# Numerical features that are marked as continuous
FLOAT_FEATURE_KEYS = ['Dew point temperature(degC)',
                       'Snowfall (cm)', 'Solar Radiation (MJ/m2)',
                       'Temperature(degC)', 'Wind speed (m/s)']
INT_FEATURE_KEYS = ['Hour', 'Visibility (10m)',
                   'Humidity(%)', 'Day', 'DayOfWeek', 'Month', 'Year']

# Feature that can be grouped into buckets
# BUCKET_FEATURE_KEYS = ['Day']

# # Number of buckets used by tf.transform for encoding each bucket feature.
# FEATURE_BUCKET_COUNT = {'age': 4}

# Feature that the model will predict
LABEL_KEY = 'Rented Bike Count'

# Utility function for renaming the feature
def transformed_name(key):
    return key + '_xf'

Overwriting bike_constants.py


In [57]:
# Set the transform module filename
_bike_transform_module_file = 'bike_transform.py'

In [59]:
%%writefile {_bike_transform_module_file}

import tensorflow as tf
import tensorflow_transform as tft

import bike_constants

# Unpack the contents of the constants module
_CATEGORICAL_FEATURE_KEYS = bike_constants.CATEGORICAL_FEATURE_KEYS
_DATE_FEATURE_KEY = bike_constants.DATE_FEATURE_KEY
_FLOAT_FEATURE_KEYS = bike_constants.FLOAT_FEATURE_KEYS
_INT_FEATURE_KEYS = bike_constants.INT_FEATURE_KEYS
# _BUCKET_FEATURE_KEYS = bike_constants.BUCKET_FEATURE_KEYS
# _FEATURE_BUCKET_COUNT = bike_constants.FEATURE_BUCKET_COUNT
_LABEL_KEY = bike_constants.LABEL_KEY
_transformed_name = bike_constants.transformed_name


# Define the transformations
def preprocessing_fn(inputs):
    """tf.transform's callback function for preprocessing inputs.
    Args:
        inputs: map from feature keys to raw not-yet-transformed features.
    Returns:
        Map from string feature key to transformed feature operations.
    """
    outputs = {}

    # Scale these features to the range [0,1]
    for key in _FLOAT_FEATURE_KEYS + _INT_FEATURE_KEYS:
        outputs[_transformed_name(key)] = tft.scale_to_0_1(inputs[key])
    
    # Bucketize these features
#     for key in _BUCKET_FEATURE_KEYS:
#         outputs[_transformed_name(key)] = tft.bucketize(inputs[key], _FEATURE_BUCKET_COUNT[key])

    # Convert strings to indices in a vocabulary
    for key in _CATEGORICAL_FEATURE_KEYS:
        outputs[_transformed_name(key)] = tft.compute_and_apply_vocabulary(inputs[key])

    # Convert the label strings to an index
    outputs[_transformed_name(_LABEL_KEY)] = tft.compute_and_apply_vocabulary(inputs[_LABEL_KEY])    

    return outputs

Overwriting bike_transform.py


You can now pass the training data, schema, and transform module to the `Transform` component. You can ignore the warning messages generated by Apache Beam regarding type hints.

In [60]:
# Ignore TF warning messages
tf.get_logger().setLevel('ERROR')

# Instantiate the Transform component
transform = tfx.components.Transform(
    examples=example_gen.outputs['examples'],
    schema=schema_gen.outputs['schema'],
    module_file=os.path.abspath(_bike_transform_module_file))

# Run the component
context.run(transform)

running bdist_wheel
running build
running build_py
creating build
creating build/lib
copying census_transform.py -> build/lib
copying traffic_transform.py -> build/lib
copying census_constants.py -> build/lib
copying bike_constants.py -> build/lib
copying bike_transform.py -> build/lib
copying add_metadata.py -> build/lib
copying testing_values.py -> build/lib
copying traffic_constants.py -> build/lib
installing to /tmp/tmp2lsg41_3
running install
running install_lib
copying build/lib/census_transform.py -> /tmp/tmp2lsg41_3
copying build/lib/traffic_transform.py -> /tmp/tmp2lsg41_3
copying build/lib/census_constants.py -> /tmp/tmp2lsg41_3
copying build/lib/bike_constants.py -> /tmp/tmp2lsg41_3
copying build/lib/bike_transform.py -> /tmp/tmp2lsg41_3
copying build/lib/add_metadata.py -> /tmp/tmp2lsg41_3
copying build/lib/testing_values.py -> /tmp/tmp2lsg41_3
copying build/lib/traffic_constants.py -> /tmp/tmp2lsg41_3
running install_egg_info
running egg_info
creating tfx_user_code_Transfo

/home/paperspace/miniconda3/envs/c2w2/lib/python3.10/site-packages/setuptools/_distutils/cmd.py:66: SetuptoolsDeprecationWarning: setup.py install is deprecated.
!!

        ********************************************************************************
        Please avoid running ``setup.py`` directly.
        Instead, use pypa/build, pypa/installer or other
        standards-based tools.

        See https://blog.ganssle.io/articles/2021/10/setup-py-deprecated.html for details.
        ********************************************************************************

!!
  self.initialize_options()


Processing ./pipeline/_wheels/tfx_user_code_Transform-0.0+d2aa9d5463897b365f6e5131a620f642a971c31592624413f0813d597f2dd603-py3-none-any.whl
Processing ./pipeline/_wheels/tfx_user_code_Transform-0.0+d2aa9d5463897b365f6e5131a620f642a971c31592624413f0813d597f2dd603-py3-none-any.whl
Processing ./pipeline/_wheels/tfx_user_code_Transform-0.0+d2aa9d5463897b365f6e5131a620f642a971c31592624413f0813d597f2dd603-py3-none-any.whl


ExecutionResult(
    component_id: Transform
    execution_id: 47
    outputs:
        transform_graph: OutputChannel(artifact_type=TransformGraph, producer_component_id=Transform, output_key=transform_graph, additional_properties={}, additional_custom_properties={}, _input_trigger=None
        transformed_examples: OutputChannel(artifact_type=Examples, producer_component_id=Transform, output_key=transformed_examples, additional_properties={}, additional_custom_properties={}, _input_trigger=None
        updated_analyzer_cache: OutputChannel(artifact_type=TransformCache, producer_component_id=Transform, output_key=updated_analyzer_cache, additional_properties={}, additional_custom_properties={}, _input_trigger=None
        pre_transform_schema: OutputChannel(artifact_type=Schema, producer_component_id=Transform, output_key=pre_transform_schema, additional_properties={}, additional_custom_properties={}, _input_trigger=None
        pre_transform_stats: OutputChannel(artifact_type=ExampleStatistics, producer_component_id=Transform, output_key=pre_transform_stats, additional_properties={}, additional_custom_properties={}, _input_trigger=None
        post_transform_schema: OutputChannel(artifact_type=Schema, producer_component_id=Transform, output_key=post_transform_schema, additional_properties={}, additional_custom_properties={}, _input_trigger=None
        post_transform_stats: OutputChannel(artifact_type=ExampleStatistics, producer_component_id=Transform, output_key=post_transform_stats, additional_properties={}, additional_custom_properties={}, _input_trigger=None
        post_transform_anomalies: OutputChannel(artifact_type=ExampleAnomalies, producer_component_id=Transform, output_key=post_transform_anomalies, additional_properties={}, additional_custom_properties={}, _input_trigger=None)

Take a peek at the `transform_graph` artifact.  It points to a directory containing three subdirectories.

In [61]:
# Get the uri of the transform graph
transform_graph_uri = transform.outputs['transform_graph'].get()[0].uri

# List the subdirectories under the uri
os.listdir(transform_graph_uri)

['transformed_metadata', 'transform_fn', 'metadata']

* The `metadata` subdirectory contains the schema of the original data.
* The `transformed_metadata` subdirectory contains the schema of the preprocessed data. 
* The `transform_fn` subdirectory contains the actual preprocessing graph. 

You can also take a look at the first three transformed examples using the helper function defined earlier.

In [62]:
# Get the URI of the output artifact representing the transformed examples
train_uri = os.path.join(transform.outputs['transformed_examples'].get()[0].uri, 'Split-train')

# Get the list of files in this directory (all compressed TFRecord files)
tfrecord_filenames = [os.path.join(train_uri, name)
                      for name in os.listdir(train_uri)]

# Create a `TFRecordDataset` to read these files
transformed_dataset = tf.data.TFRecordDataset(tfrecord_filenames, compression_type="GZIP")

In [63]:
# Get 3 records from the dataset
sample_records_xf = get_records(transformed_dataset, 3)

# Print the output
pp.pprint(sample_records_xf)

[{'features': {'feature': {'Dew point temperature(degC)_xf': {'floatList': {'value': [0.21265379]}},
                           'Functioning Day_xf': {'int64List': {'value': ['0']}},
                           'Holiday_xf': {'int64List': {'value': ['0']}},
                           'Hour_xf': {'floatList': {'value': [0.0]}},
                           'Humidity(%)_xf': {'floatList': {'value': [0.37755102]}},
                           'Rented Bike Count_xf': {'int64List': {'value': ['142']}},
                           'Seasons_xf': {'int64List': {'value': ['2']}},
                           'Snowfall (cm)_xf': {'floatList': {'value': [0.0]}},
                           'Solar Radiation (MJ/m2)_xf': {'floatList': {'value': [0.0]}},
                           'Temperature(degC)_xf': {'floatList': {'value': [0.22027971]}},
                           'Visibility (10m)_xf': {'floatList': {'value': [1.0]}},
                           'Wind speed (m/s)_xf': {'floatList': {'value': [0.301369